In [1]:
from datasets import load_dataset
import numpy as np

In [2]:
trainData = load_dataset(
    'imagenet-1k',
    split='train',
    use_auth_token=True,
    cache_dir="/scratch/doa240/.cache/huggingface/datasets",
    streaming=True,
    ignore_verifications=True  # set to True if seeing splits Error
)
trainData = trainData.shuffle(seed=101)

In [3]:
X_train = []
y_train = []

i = 1
for data in trainData:
    img = data['image']
    img = img.resize((224,224))
    img = img.convert('L')
    img = np.array(img)
    if(len(X_train) > 0 and img.shape != X_train[-1].shape):
        print(i, "wrong", img.shape, X_train[-1].shape)
        break
    X_train.append(img)
    y_train.append(data['label'])
    if i % 1000 == 0:
        print(i)
    if i == 150000:
        break
    i = i+1

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000


In [4]:
x_train = np.array(X_train)
x_train = x_train.reshape(x_train.shape[0], -1)
print(x_train.shape)

(150000, 50176)


In [5]:
y_train = np.array(y_train)
print(y_train.shape)

(150000,)


In [6]:
import autosklearn.classification
import autosklearn.metrics
from pprint import pprint
import joblib
from smac.tae import StatusType 
from sklearn.metrics import accuracy_score, f1_score 

In [7]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=(3600*24), 
    per_run_time_limit = (3600*24)/6,
    seed=101,
    n_jobs=-1,
    memory_limit= 3072*100
)
automl.fit(x_train, y_train)

[WARNING] [2023-04-04 02:28:42,376:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 02:34:17,038:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 02:39:40,226:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 02:42:25,858:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 02:44:17,527:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 02:55:34,791:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 03:10:46,052:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-04-04 05:00:27,152:Client-EnsembleBuilder] No models better than random - using Dummy losses!
	Models besides current dummy model: 0
	Dummy models: 1
[WARNING] [2023-04-04 06:23:06,537:Client-EnsembleBuilder] No models better th

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      memory_limit=307200, n_jobs=-1,
                      per_run_time_limit=14400.0, seed=101,
                      time_left_for_this_task=86400)

In [8]:
pprint(automl.sprint_statistics())

('auto-sklearn results:\n'
 '  Dataset name: 14a0a2ba-d269-11ed-8a71-00110a6b78e8\n'
 '  Metric: accuracy\n'
 '  Best validation score: 0.011455\n'
 '  Number of target algorithm runs: 158\n'
 '  Number of successful target algorithm runs: 43\n'
 '  Number of crashed target algorithm runs: 49\n'
 '  Number of target algorithms that exceeded the time limit: 61\n'
 '  Number of target algorithms that exceeded the memory limit: 5\n')


In [9]:
print(automl.leaderboard())

          rank  ensemble_weight type      cost      duration
model_id                                                    
131          1             0.02  lda  0.988545  11277.734571
133          2             0.02  lda  0.988788   2839.181123
126          3             0.02  lda  0.988808   2816.876561
122          4             0.02  lda  0.988929   2565.308329
149          5             0.02  lda  0.989293   4303.493482
123          6             0.04  lda  0.990081   2428.859432
148          7             0.04  lda  0.990263   2780.847593
143          8             0.04  lda  0.990667   2856.428433
140          9             0.02  lda  0.991717   3803.959288
66          10             0.02  lda  0.991818   3449.527500
110         11             0.02  lda  0.991919   2850.401667
43          12             0.02  lda  0.992000   3489.962146
107         13             0.02  lda  0.992081   3087.108957
61          14             0.02  lda  0.992141   3226.186872
71          15          

In [10]:
print(automl.show_models())

{39: {'model_id': 39, 'rank': 1, 'cost': 0.9927878787878788, 'ensemble_weight': 0.02, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x1550ea416700>, 'balancing': Balancing(random_state=101, strategy='weighting'), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x155312b468e0>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x1551433ffac0>, 'sklearn_classifier': LinearDiscriminantAnalysis(tol=0.0004558809131391089)}, 43: {'model_id': 43, 'rank': 2, 'cost': 0.992, 'ensemble_weight': 0.02, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x155312205370>, 'balancing': Balancing(random_state=101, strategy='weighting'), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x155312e9ce20>, 'classifier': <autos

In [11]:
joblib.dump(automl, "imagenetOriginal.joblib")

['imagenetOriginal.joblib']

In [12]:
testData = load_dataset(
    'imagenet-1k',
    split='validation',
    use_auth_token=True,
    cache_dir="/scratch/doa240/.cache/huggingface/datasets",
    streaming=True,
    ignore_verifications=True  # set to True if seeing splits Error
)

In [13]:
X_test = []
y_test = []

for data in testData:
    img = data['image']
    img = img.resize((224,224))
    img = img.convert('L')
    img = np.array(img)
    X_test.append(img)
    y_test.append(data['label'])

In [14]:
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], -1)
print(X_test.shape)

(50000, 50176)


In [15]:
y_test = np.array(y_test)
print(y_test.shape)

(50000,)


In [16]:
# evaluate
pred = automl.predict(X_test)
test_acc = accuracy_score(y_test, pred)
print("Test Accuracy score {0}".format(test_acc))

Test Accuracy score 0.01204


In [17]:
classes = [0] * 1000
classesCnt = [0] * 1000

for i in range(len(pred)):
    if pred[i] == y_test[i]:
        classes[pred[i]] += 1
    classesCnt[y_test[i]] += 1
    
for i in range(1000):
    print("Label:", i, classes[i]/classesCnt[i])

Label: 0 0.02
Label: 1 0.0
Label: 2 0.02
Label: 3 0.02
Label: 4 0.06
Label: 5 0.0
Label: 6 0.06
Label: 7 0.0
Label: 8 0.02
Label: 9 0.0
Label: 10 0.0
Label: 11 0.0
Label: 12 0.0
Label: 13 0.02
Label: 14 0.0
Label: 15 0.0
Label: 16 0.0
Label: 17 0.0
Label: 18 0.0
Label: 19 0.0
Label: 20 0.0
Label: 21 0.0
Label: 22 0.02
Label: 23 0.0
Label: 24 0.0
Label: 25 0.0
Label: 26 0.0
Label: 27 0.0
Label: 28 0.0
Label: 29 0.04
Label: 30 0.0
Label: 31 0.02
Label: 32 0.0
Label: 33 0.0
Label: 34 0.06
Label: 35 0.0
Label: 36 0.0
Label: 37 0.0
Label: 38 0.0
Label: 39 0.0
Label: 40 0.0
Label: 41 0.0
Label: 42 0.0
Label: 43 0.0
Label: 44 0.0
Label: 45 0.0
Label: 46 0.0
Label: 47 0.0
Label: 48 0.0
Label: 49 0.0
Label: 50 0.0
Label: 51 0.0
Label: 52 0.0
Label: 53 0.0
Label: 54 0.0
Label: 55 0.0
Label: 56 0.0
Label: 57 0.02
Label: 58 0.0
Label: 59 0.0
Label: 60 0.0
Label: 61 0.0
Label: 62 0.0
Label: 63 0.0
Label: 64 0.0
Label: 65 0.0
Label: 66 0.0
Label: 67 0.0
Label: 68 0.02
Label: 69 0.0
Label: 70 0.0
Lab

In [18]:
classes = np.array(classes)
print(np.sum(classes)/len(y_test))

0.01204
